In [2]:
import sys
!"{sys.executable}" -m pip install netCDF4 scipy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Shantanoo Aher\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



  Using cached netCDF4-1.7.2-cp312-cp312-win_amd64.whl.metadata (1.8 kB)
  Using cached cftime-1.6.4.post1-cp312-cp312-win_amd64.whl.metadata (8.9 kB)
Using cached netCDF4-1.7.2-cp312-cp312-win_amd64.whl (7.0 MB)
Using cached cftime-1.6.4.post1-cp312-cp312-win_amd64.whl (178 kB)


In [3]:
import os
import numpy as np
from glob import glob
from datetime import datetime
import xarray as xr

In [4]:
npy_folder=r"C:\Users\Shantanoo Aher\OneDrive\Documents\npyf"
nc4_output="INSAT3DR_Stacked.nc4"
band_order=['WV','VIS','SWIR','TIR1','TIR2']
band_to_index = {b: i for i, b in enumerate(band_order)}

In [5]:
npy_files = sorted(glob(os.path.join(npy_folder, "*.npy")))

#Parsing timestamps and unique bands
time_strings = set()
for f in npy_files:
    parts = os.path.basename(f).split("_")
    if len(parts) >= 3:
        time_strings.add(f"{parts[1]}_{parts[2]}")
time_strings = sorted(time_strings)

In [6]:
#Converting to proper datetime
def parse_time(ts):
    return datetime.strptime(ts, "%d%b%Y_%H%M")

time_objects = [parse_time(t) for t in time_strings]
shape_counts = {}
for f in npy_files:
    shape = np.load(f).shape
    shape_counts[shape] = shape_counts.get(shape, 0) + 1

ref_shape = max(shape_counts, key=shape_counts.get)
nlat, nlon = ref_shape
ntimes = len(time_objects)
nbands = len(band_order)

In [7]:
stack = np.full((ntimes, nbands, nlat, nlon), np.nan, dtype=np.float32)

for f in npy_files:
    fname = os.path.basename(f)
    parts = fname.split("_")
    if len(parts) < 4:
        print(f"Skipping malformed filename: {fname}")
        continue

    timestamp = f"{parts[1]}_{parts[2]}"
    band = parts[-1].replace(".npy", "").upper()

    if band not in band_to_index:
        print(f"Skipping unknown band: {band}")
        continue

    if timestamp not in time_strings:
        print(f"Skipping unknown timestamp: {timestamp}")
        continue

    data = np.load(f)
    if data.shape != ref_shape:
        print(f"Skipping file with mismatched shape: {fname} → {data.shape}")
        continue

    time_idx = time_strings.index(timestamp)
    band_idx = band_to_index[band]
    stack[time_idx, band_idx] = data

    

In [8]:
lat_vals=np.load("lat_vals.npy")
lon_vals=np.load("lon_vals.npy")

In [9]:
ds = xr.Dataset(
    data_vars={
        "BT": (("time", "band", "lat", "lon"), stack)
    },
    coords={
        "time": time_objects,
        "band": band_order,
        "lat": lat_vals,
        "lon": lon_vals,
    },
    attrs={
        "title": "INSAT-3DR Brightness Temperature Dataset",
        "description": "Stacked BT values from cleaned and georeferenced GeoTIFFs",
        "institution": "Bharatiya Antariksh Hackathon 2025",
        "source": "INSAT-3DR VHRR Radiance Converted to BT",
    }
)

In [10]:
ds.to_netcdf(nc4_output, format="NETCDF4",engine='netcdf4')